# Model Time Series for COVID: Publish for app

This notebook use multistep time serie model  (predicting number of cases future next days).

It converts Tensorflow model into TensorFlow Lite to be able to use it in a Lambda fonction on AWS.

After that, the lite model is tested and publish on AWS

Finally, the lambda function is tested

## Convert model in TFlite

**IMPORTANT TO DO manually:**
- **Update TRAIN_SPLIT in my_helpers.model module**

### import

In [1]:
# data useful lib
import pandas as pd
import numpy as np

# helper lib
import shutil
import os, stat
import re
import datetime
import math

# read json from http
import json
import urllib.request

# read csv from http
import io
import requests

# model lib
import tensorflow as tf

# from project
from my_helpers.model import prepare_to_lambda, retrieve_from_lambda
from my_helpers.model import prepare_to_lambda_future
from my_helpers.model import create_list_past_hist, predict_list

### Definitions

In [2]:
PATH_TO_SAVE_DATA = "."
PATH_DF_POS_FR = PATH_TO_SAVE_DATA + '/' + 'df_pos_fr.csv'
PATH_DF_TEST_FR = PATH_TO_SAVE_DATA + '/' + 'df_test_fr.csv'
PATH_JSON_METEO_FR = PATH_TO_SAVE_DATA + '/' + 'data_meteo_fr.json'
PATH_DF_FEAT_FR = PATH_TO_SAVE_DATA + '/' + 'df_feat_fr.csv' 
PATH_GEO_DEP_FR = PATH_TO_SAVE_DATA + '/sources/geofrance/' + 'departments.csv'
PATH_MDL_MULTI_STEP = PATH_TO_SAVE_DATA + '/' + "mdl_multi_step_pos_fr_tcn"
PATH_MDL_MULTI_TFLITE = PATH_TO_SAVE_DATA + '/' + \
    'serverless/tensorflow_lite_on_aws_lambda'
PATH_MDL_MULTI_TFLITE_FILE = PATH_MDL_MULTI_TFLITE + '/' + \
    "converted_model.tflite"
PATH_SERVERLESS = PATH_MDL_MULTI_TFLITE + '/' + 'serverless.yml'

date_format = "%Y-%m-%d"

#NB_POS_DATE_MIN_DF_FEAT = 140734 # on 13/05/2020
NB_POS_DATE_MIN_DF_FEAT = 140227 # on 12/05/2020

URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer'

# model 
PAST_HISTORY = 14 # days used to predict next values in future
FUTURE_TARGET = 7 # nb predict days later
STEP = 1

# plot
NB_DAY_PLOT = FUTURE_TARGET*9

# train split
from my_helpers.model import TRAIN_SPLIT
print(f"TRAIN_SPLIT = {TRAIN_SPLIT}")

TRAIN_SPLIT = 272


### helper functions

In [3]:
# save file before update
def clean_file(path_file_name):
    '''
    Clean file already traited : rename file with date
    '''
    try:
        d = datetime.datetime.now()
        str_date = '_' + d.strftime("%Y%m%d_%H_%M_%S")
       
        res_re = re.search('\.\w+$', path_file_name)
        
        path_file_name_saved = \
            path_file_name[0:res_re.start()] + str_date + res_re.group(0)
         
        shutil.move(path_file_name, path_file_name_saved) 
        print('File {} moved!'.format(path_file_name_saved))
    except:
        print('File {} does not exist!'.format(path_file_name))

### load data

In [4]:
df_feat_fr = pd.read_csv(PATH_DF_FEAT_FR)
df_feat_fr.index = df_feat_fr["date"]
df_feat_fr["train"] = [True if I <= TRAIN_SPLIT else False \
                       for I in range(df_feat_fr.shape[0])]
df_feat_fr


,T_min,date,T_max,H_min,H_max,extrap,pos,age_pos,test,age_test,day_num,nb_cases,sum_cases,Rt,rate_pos,train
date,,,,,,,,,,,,,,,,
2020-05-13,284.926667,2020-05-13,290.505000,64.661017,88.135593,0.0,881,61.023837,39048,55.435003,3,141108,NaN,NaN,2.256198,True
2020-05-14,285.050000,2020-05-14,290.963333,59.406780,84.847458,0.0,979,60.426966,41986,54.831611,4,142087,NaN,NaN,2.331730,True
2020-05-15,285.308333,2020-05-15,291.920000,57.372881,82.966102,0.0,1021,60.042116,47686,54.385124,5,143108,NaN,NaN,2.141090,True
2020-05-16,284.956667,2020-05-16,293.500000,53.741379,86.534483,0.0,291,60.158076,16535,54.486302,6,143399,NaN,NaN,1.759903,True
2020-05-17,285.598333,2020-05-17,294.446667,49.879310,85.500000,0.0,139,61.568345,6604,58.291036,0,143538,NaN,NaN,2.104785,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-18,284.926667,2021-02-18,290.505000,64.661017,88.135593,1.0,23249,45.952772,365658,45.595477,4,3513658,263845.0,0.924925,6.358127,False
2021-02-19,284.926667,2021-02-19,290.505000,64.661017,88.135593,1.0,25239,45.428226,407099,45.506624,5,3538897,265734.0,0.934712,6.199720,False
2021-02-20,284.926667,2021-02-20,290.505000,64.661017,88.135593,1.0,14158,45.102698,215229,45.446264,6,3553055,266867.0,0.940467,6.578110,False


### prepare features

In [5]:
features = df_feat_fr.copy().filter(items=['T_min', 'T_max', 'H_min',
                                           'H_max', 'pos', 'test', 'day_num',
                                          'age_pos', 'age_test'])
features

,T_min,T_max,H_min,H_max,pos,test,day_num,age_pos,age_test
date,,,,,,,,,
2020-05-13,284.926667,290.505000,64.661017,88.135593,881,39048,3,61.023837,55.435003
2020-05-14,285.050000,290.963333,59.406780,84.847458,979,41986,4,60.426966,54.831611
2020-05-15,285.308333,291.920000,57.372881,82.966102,1021,47686,5,60.042116,54.385124
2020-05-16,284.956667,293.500000,53.741379,86.534483,291,16535,6,60.158076,54.486302
2020-05-17,285.598333,294.446667,49.879310,85.500000,139,6604,0,61.568345,58.291036
...,...,...,...,...,...,...,...,...,...
2021-02-18,284.926667,290.505000,64.661017,88.135593,23249,365658,4,45.952772,45.595477
2021-02-19,284.926667,290.505000,64.661017,88.135593,25239,407099,5,45.428226,45.506624
2021-02-20,284.926667,290.505000,64.661017,88.135593,14158,215229,6,45.102698,45.446264


### Prepare dataset

In [6]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)
dataset = (dataset-data_mean)/data_std

In [7]:
dataset.shape[1]

9

In [8]:
PAST_HISTORY

14

In [9]:
dataset.shape[1]

9

### Load model

In [10]:
%%time
# reload best model
multi_step_model = tf.keras.models.load_model(PATH_MDL_MULTI_STEP)

/anaconda3/envs/coronavirusModel/lib/python3.7/site-packages/tensorflow/python/keras/layers/core.py:1045: UserWarning: tcn.tcn is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


CPU times: user 2.47 s, sys: 182 ms, total: 2.65 s
Wall time: 4.71 s


In [11]:
multi_step_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 14, 9)]           0         
_________________________________________________________________
tcn (TCN)                    (None, 64)                43136     
_________________________________________________________________
dense (Dense)                (None, 7)                 455       
Total params: 43,591
Trainable params: 43,591
Non-trainable params: 0
_________________________________________________________________


In [12]:
multi_step_model.inputs[0].dtype

tf.float32

In [13]:
run_model = tf.function(lambda x: multi_step_model(x))
# This is important, let's fix the input size.
INPUT_SIZE = dataset.shape[1]
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1, PAST_HISTORY, INPUT_SIZE],
                  multi_step_model.inputs[0].dtype))

# model directory.
MODEL_DIR = PATH_TO_SAVE_DATA + "/" + "keras_tcn"
multi_step_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

'''converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.allow_custom_ops=True'''

tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./keras_tcn/assets


### Save model TFlite

In [14]:
clean_file(PATH_MDL_MULTI_TFLITE_FILE)

File ./serverless/tensorflow_lite_on_aws_lambda/converted_model_20210226_15_21_27.tflite moved!


In [17]:
PATH_MDL_MULTI_TFLITE_FILE

'./serverless/tensorflow_lite_on_aws_lambda/converted_model.tflite'

In [15]:
open(PATH_MDL_MULTI_TFLITE_FILE, "wb").write(tflite_model)

196672

## Test converted model

### Predict with TF model (not-converted one)

#### Past days

In [16]:
# TENSORFLOW MODEL :
# prepare list of past histories
list_x = create_list_past_hist(dataset)
# predict
y_multi_pred = predict_list(list_x, multi_step_model)
# convert in positive cases
y_pos_pred = (y_multi_pred * data_std[4]) + data_mean[4] 
y_pos_pred

[209 - 223]
[216 - 230]
[223 - 237]
[230 - 244]
[237 - 251]
[244 - 258]
[251 - 265]
[258 - 272]
[265 - 279]
9


array([[20538.51  , 18479.293 , 13108.777 ,  5685.08  ,  9875.467 ,
         1628.1807, 22780.219 , 21162.504 , 19893.139 , 16835.629 ,
         3569.0166, 13924.408 ,  3121.7744, 29824.973 , 23517.23  ,
        21918.418 , 19885.516 , 20503.314 , 13237.41  ,  3887.3198,
        30054.156 , 22836.047 , 21899.05  , 22845.64  , 22575.371 ,
        14757.234 ,  2076.4922, 31194.574 , 24886.79  , 22802.73  ,
        25516.66  , 25100.79  , 14031.648 ,  3924.566 , 32506.375 ,
        25113.79  , 23931.66  , 24974.508 , 24851.205 , 14354.719 ,
         3184.6895, 33211.97  , 26777.797 , 22094.45  , 22528.61  ,
        23413.355 , 13224.197 ,  2137.3262, 29775.21  , 27565.96  ,
        18637.902 , 27735.988 , 22670.617 , 12462.65  ,  1566.5254,
        33615.902 , 25530.336 , 19962.438 , 22010.379 , 21153.875 ,
        10770.547 ,  2435.4277, 30202.438 ]], dtype=float32)

#### Future days

In [18]:
# prepare data : very last days
x_for_future = np.array([dataset[-PAST_HISTORY:,:]]) 
# predict next days
y_future_pred = multi_step_model.predict(x_for_future)

### Predict with TFlite & Compare 

#### Past days

In [19]:
# CONVERTED LITE MODEL
# load 
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    # predict with tensorflow model
    expected = multi_step_model.predict(x_multi)
    # predict with TFlite model
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Reload Tlite model

In [20]:
interpreter = tf.lite.Interpreter(model_path=PATH_MDL_MULTI_TFLITE_FILE)

In [21]:
PATH_MDL_MULTI_TFLITE_FILE

'./serverless/tensorflow_lite_on_aws_lambda/converted_model.tflite'

### Predict reloaded model

#### Past days

In [22]:
# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    
    expected = multi_step_model.predict(x_multi)
    
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [23]:
x_multi.shape

(1, 14, 9)

In [24]:
len(list_x)

9

## Update API lambda AWS

### API lambda simulate

#### Past days

In [25]:
dataset.shape

(286, 9)

In [26]:
json_list_list_x = prepare_to_lambda(dataset)
# simulate input to lambda (double dumps ? why ? i don't know yet)
json_list_list_x = json.dumps(json_list_list_x)
# simulate lambda

event = {"body": json_list_list_x}

[209 - 223]
[216 - 230]
[223 - 237]
[230 - 244]
[237 - 251]
[244 - 258]
[251 - 265]
[258 - 272]
[265 - 279]


In [27]:
# lambda code (file ./serverless/tensorflow-lite-on-aws-lambda/handler.py)
from serverless.tensorflow_lite_on_aws_lambda.handler import predict
context = None
response = predict(event, context)


INPUT : nb. arrays : 9 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 9 / arrays shape in list: (1, 7)


In [28]:
# Retrieve from lambda in App code
# input : response
y_multi_pred_out = retrieve_from_lambda(response)      
y_multi_pred_out.shape

(1, 63)

In [29]:
y_multi_pred

array([[ 0.63133883,  0.48457733,  0.10181797, -0.42727283, -0.12862182,
        -0.71641016,  0.7911065 ,  0.67581123,  0.5853429 ,  0.3674326 ,
        -0.5780858 ,  0.15994838, -0.6099609 ,  1.2931899 ,  0.8436339 ,
         0.72968566,  0.5847996 ,  0.62883043,  0.11098569, -0.55540013,
         1.3095238 ,  0.7950855 ,  0.7283052 ,  0.79576933,  0.7765069 ,
         0.21930435, -0.68445873,  1.3908021 ,  0.9412429 ,  0.79271096,
         0.9861343 ,  0.9564949 ,  0.16759148, -0.5527456 ,  1.4842949 ,
         0.95742136,  0.87317044,  0.94749475,  0.9387069 ,  0.19061682,
        -0.605477  ,  1.5345829 ,  1.0760161 ,  0.7422315 ,  0.77317417,
         0.8362305 ,  0.11004395, -0.6801231 ,  1.2896433 ,  1.1321889 ,
         0.49588153,  1.1443069 ,  0.78329515,  0.05576815, -0.72080433,
         1.5633714 ,  0.9871089 ,  0.5902819 ,  0.7362398 ,  0.6751962 ,
        -0.06482896, -0.65887725,  1.320092  ]], dtype=float32)

In [30]:
y_multi_pred_out

array([[ 0.63133878,  0.48457727,  0.10181809, -0.42727283, -0.12862167,
        -0.71641016,  0.79110658,  0.67581141,  0.58534282,  0.36743265,
        -0.5780859 ,  0.15994829, -0.60996068,  1.29318988,  0.84363413,
         0.7296856 ,  0.58480006,  0.62883031,  0.11098542, -0.55540049,
         1.30952406,  0.79508561,  0.72830504,  0.79576945,  0.77650714,
         0.21930435, -0.68445814,  1.39080191,  0.94124293,  0.79271096,
         0.98613417,  0.95649457,  0.16759196, -0.5527457 ,  1.48429525,
         0.95742089,  0.8731705 ,  0.94749457,  0.93870687,  0.19061694,
        -0.60547674,  1.53458285,  1.07601607,  0.74223155,  0.77317429,
         0.83623052,  0.11004371, -0.68012321,  1.28964329,  1.13218868,
         0.49588159,  1.14430749,  0.78329492,  0.05576774, -0.72080457,
         1.56337154,  0.98710912,  0.59028202,  0.73623961,  0.67519617,
        -0.06482909, -0.65887713,  1.32009113]])

In [31]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### Future days

In [32]:
# Prepare data to lambda (future)
json_list_list_x = prepare_to_lambda_future(dataset)

# simulate lambda
json_list_list_x = json.dumps(json_list_list_x) # dumps again : I dont know why
event = {"body": json_list_list_x}
context = None
response = predict(event, context)
y_future_pred_out = retrieve_from_lambda(response)      
y_future_pred_out.shape

INPUT : nb. arrays : 1 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 1 / arrays shape in list: (1, 7)


(1, 7)

In [33]:
y_future_pred_out

array([[ 1.18119955,  0.76872414,  0.94762379,  1.00058341,  0.05631598,
        -0.75218713,  1.60633397]])

In [34]:
y_future_pred

array([[ 1.1811993 ,  0.7687239 ,  0.94762385,  1.0005833 ,  0.05631609,
        -0.752187  ,  1.6063344 ]], dtype=float32)

In [35]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Update AWS Lambda with new model


This part does:
- Go to : ./serverless//tensorflow-lite-on-aws-lambda
    
- Execute : sls deploy -v

In [36]:
str_exe = '#!/bin/bash\n' + \
    'export PATH="/usr/local/bin:$PATH"\n' + \
    f'cd {PATH_MDL_MULTI_TFLITE}\n' + \
    'serverless deploy -v'
str_exe

'#!/bin/bash\nexport PATH="/usr/local/bin:$PATH"\ncd ./serverless/tensorflow_lite_on_aws_lambda\nserverless deploy -v'

In [37]:
open('deploy_serverless.sh', "w").write(str_exe)
os.chmod('deploy_serverless.sh', stat.S_IRWXU)

In [38]:
!cat ./deploy_serverless.sh

#!/bin/bash
export PATH="/usr/local/bin:$PATH"
cd ./serverless/tensorflow_lite_on_aws_lambda
serverless deploy -v

In [40]:
!./deploy_serverless.sh

Serverless: Generated requirements from /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda/requirements.txt in /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda/.serverless/requirements.txt...
Serverless: Installing requirements from /Users/gregory/Library/Caches/serverless-python-requirements/ef4e42eb03bbad46f74fee99a3c01f994e8119e7a557f947779093ab37b248d3_slspyc/requirements.txt ...
Serverless: Docker Image: lambci/lambda:build-python3.7
Serverless: Using download cache directory /Users/gregory/Library/Caches/serverless-python-requirements/downloadCacheslspyc
Serverless: Running docker run --rm -v /Users/gregory/Library/Caches/serverless-python-requirements/ef4e42eb03bbad46f74fee99a3c01f994e8119e7a557f947779093ab37b248d3_slspyc\:/var/task\:z -v /Users/gregory/Library/Caches/serverless-python-requirements/dow

### API AWS real Test

#### Past days

In [41]:
# prepare input
json_list_list_x = prepare_to_lambda(dataset)
# REQUEST
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

[209 - 223]
[216 - 230]
[223 - 237]
[230 - 244]
[237 - 251]
[244 - 258]
[251 - 265]
[258 - 272]
[265 - 279]
status code :  200
[[[0.6313385963439941, 0.4845772087574005, 0.10181833058595657, -0.42727288603782654, -0.1286216676235199, -0.7164102792739868, 0.7911065816879272]], [[0.6758114695549011, 0.5853428244590759, 0.3674325942993164, -0.5780857801437378, 0.15994828939437866, -0.6099607944488525, 1.2931898832321167]], [[0.8436340689659119, 0.7296856045722961, 0.5848000049591064, 0.6288303136825562, 0.1109854206442833, -0.5554006099700928, 1.3095240592956543]], [[0.795085608959198, 0.728304922580719, 0.7957692742347717, 0.7765070199966431, 0.21930435299873352, -0.6844580173492432, 1.3908019065856934]], [[0.9412429928779602, 0.7927110195159912, 0.9861341714859009, 0.9564946889877319, 0.16759172081947327, -0.5527455806732178, 1.4842952489852905]], [[0.9574208855628967, 0.8731704950332642, 0.947494387626648, 0.9387067556381226, 0.19061687588691711, -0.6054768562316895, 1.5345828533172607

In [42]:
len(json_list_list_x)

23640

In [43]:
resp.json()

[[[0.6313385963439941,
   0.4845772087574005,
   0.10181833058595657,
   -0.42727288603782654,
   -0.1286216676235199,
   -0.7164102792739868,
   0.7911065816879272]],
 [[0.6758114695549011,
   0.5853428244590759,
   0.3674325942993164,
   -0.5780857801437378,
   0.15994828939437866,
   -0.6099607944488525,
   1.2931898832321167]],
 [[0.8436340689659119,
   0.7296856045722961,
   0.5848000049591064,
   0.6288303136825562,
   0.1109854206442833,
   -0.5554006099700928,
   1.3095240592956543]],
 [[0.795085608959198,
   0.728304922580719,
   0.7957692742347717,
   0.7765070199966431,
   0.21930435299873352,
   -0.6844580173492432,
   1.3908019065856934]],
 [[0.9412429928779602,
   0.7927110195159912,
   0.9861341714859009,
   0.9564946889877319,
   0.16759172081947327,
   -0.5527455806732178,
   1.4842952489852905]],
 [[0.9574208855628967,
   0.8731704950332642,
   0.947494387626648,
   0.9387067556381226,
   0.19061687588691711,
   -0.6054768562316895,
   1.5345828533172607]],
 [[1.07601

In [44]:
y_multi_pred_out = retrieve_from_lambda(resp)      
y_multi_pred_out.shape

(1, 63)

In [45]:
y_multi_pred_out

array([[ 0.6313386 ,  0.48457721,  0.10181833, -0.42727289, -0.12862167,
        -0.71641028,  0.79110658,  0.67581147,  0.58534282,  0.36743259,
        -0.57808578,  0.15994829, -0.60996079,  1.29318988,  0.84363407,
         0.7296856 ,  0.5848    ,  0.62883031,  0.11098542, -0.55540061,
         1.30952406,  0.79508561,  0.72830492,  0.79576927,  0.77650702,
         0.21930435, -0.68445802,  1.39080191,  0.94124299,  0.79271102,
         0.98613417,  0.95649469,  0.16759172, -0.55274558,  1.48429525,
         0.95742089,  0.8731705 ,  0.94749439,  0.93870676,  0.19061688,
        -0.60547686,  1.53458285,  1.07601619,  0.74223149,  0.77317429,
         0.8362304 ,  0.11004368, -0.68012321,  1.28964329,  1.13218868,
         0.49588153,  1.14430761,  0.7832948 ,  0.0557678 , -0.72080457,
         1.56337142,  0.98710924,  0.59028202,  0.73623967,  0.67519617,
        -0.06482909, -0.65887702,  1.32009125]])

In [46]:
y_multi_pred

array([[ 0.63133883,  0.48457733,  0.10181797, -0.42727283, -0.12862182,
        -0.71641016,  0.7911065 ,  0.67581123,  0.5853429 ,  0.3674326 ,
        -0.5780858 ,  0.15994838, -0.6099609 ,  1.2931899 ,  0.8436339 ,
         0.72968566,  0.5847996 ,  0.62883043,  0.11098569, -0.55540013,
         1.3095238 ,  0.7950855 ,  0.7283052 ,  0.79576933,  0.7765069 ,
         0.21930435, -0.68445873,  1.3908021 ,  0.9412429 ,  0.79271096,
         0.9861343 ,  0.9564949 ,  0.16759148, -0.5527456 ,  1.4842949 ,
         0.95742136,  0.87317044,  0.94749475,  0.9387069 ,  0.19061682,
        -0.605477  ,  1.5345829 ,  1.0760161 ,  0.7422315 ,  0.77317417,
         0.8362305 ,  0.11004395, -0.6801231 ,  1.2896433 ,  1.1321889 ,
         0.49588153,  1.1443069 ,  0.78329515,  0.05576815, -0.72080433,
         1.5633714 ,  0.9871089 ,  0.5902819 ,  0.7362398 ,  0.6751962 ,
        -0.06482896, -0.65887725,  1.320092  ]], dtype=float32)

In [47]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### future days

In [48]:
# prepare input
json_list_list_x = prepare_to_lambda_future(dataset)
# REQUEST URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer' 
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

status code :  200
[[[1.1811994314193726, 0.7687239646911621, 0.9476237297058105, 1.000583529472351, 0.05631598085165024, -0.7521871328353882, 1.6063339710235596]]]


In [49]:
y_future_pred_out = retrieve_from_lambda(resp)      
y_future_pred_out

array([[ 1.18119943,  0.76872396,  0.94762373,  1.00058353,  0.05631598,
        -0.75218713,  1.60633397]])

In [50]:
y_future_pred

array([[ 1.1811993 ,  0.7687239 ,  0.94762385,  1.0005833 ,  0.05631609,
        -0.752187  ,  1.6063344 ]], dtype=float32)

In [51]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.
